# Entendendo regionais de saúde

## Contexto

Existe mais de uma classificação de regiões de saúde. Segundo o Ministério da Saúde:

> "O Sistema Único de Saúde (SUS) é definido, na Constituição Federal, como um conjunto de ações e serviços de saúde, organizado em rede regionalizada e hierarquizada, onde o direito à saúde deve se efetivar, por meio da implementação das políticas de saúde. O Decreto nº 7.508, em junho de 2011, regulamentou a Lei nº 8.080/90 e **definiu Região de Saúde como o “espaço geográfico contínuo constituído por agrupamento de municípios limítrofes, delimitado a partir de identidades culturais, econômicas e sociais e de redes de comunicação e infraestrutura de transportes compartilhados, com a finalidade de integrar a organização, o planejamento e a execução de ações e serviços de saúde**. Institui, ainda, as Comissões Intergestores como as instâncias de pactuação consensual entre os entes federativos para definição das regras da gestão compartilhada do SUS, a partir da organização e do funcionamento das ações e serviços de saúde integrados em redes de atenção à saúde, sendo no âmbito nacional, a Comissão Intergestores Tripartite (CIT); no âmbito estadual, a Comissão Intergestores Bipartite (CIB); no âmbito regional, a Comissão Intergestores Regional, reconhecidas pela Lei nº 12.466/11."

Lindo, mas dentre as regiões existem outras divisões de: Departamentos Regionais de Saúde, Redes Regionais de Atenção à Saúde (caso de SP), etc - que não necessariamente se respeitam (vide imagem).

<!-- ![image.png](attachment:image.png=100x20) -->

<img src="regional-sp.png" alt="drawing" width="500"/>


## Dados

- [SAGE (used by IEPS)](https://sage.saude.gov.br/?link=paineis/regiaoSaude/corpao&flt=false&param=null&ufibge=&municipioibge=&cg=&tc=&re_giao=&rm=&qs=&idPagina=83#): *lista_dados_sage.csv*
- [MS](https://www.saude.gov.br/gestao-do-sus/articulacao-interfederativa/827-articulacao-interfederativa/40853-regionalizacao): *CIR-em-20-01-17.csv*
- [Ours](http://45.55.43.231:7000/br/places/ids): from API, based on *lista de regionais de saude.xlsx* (Lari)

## Descobertas

- Nossas regiões hoje são 358, enquanto no SAGE existem 450 e no MS 437!
- Somente MG e DF são diferentes no SAGE e MS: 
  - MG tem 89 regiões no MS e 77 no SAGE - no nosso, tem 28 regiões
  - DF tem uma regional no SAGE, enquanto no MS não tem regional - nosso também é 1 região
  
### E agora, o que tá certo?

Alinhados com MS & SAGE:
- [AC](https://www.conass.org.br/pdf/planos-estaduais-de-saude/AC_PES%202016-2019%20-%20SESACRE-%20FINAL.pdf)

> A regionalização é a diretriz do Sistema Único de Saúde – SUS que orienta o processo
de descentralização das ações e serviços de saúde e os processos de negociação e pactuação
entre entes públicos. No Estado do Acre está representada por **03 regiões de saúde: Baixo Acre
e Purus, Alto Acre e Juruá/Tarauacá-Envira** e uma macrorregião de saúde tendo como
município sede, Rio Branco"

- [AL](http://cidadao.saude.al.gov.br/informacoes/guias-dos-municipios/)
- [AM](https://cosemsam.org.br/area-tecnica/municipios-do-amazonas/medio-amazonas)

Alinhados com o nosso:
- [SP](http://www.saude.sp.gov.br/ses/institucional/departamentos-regionais-de-saude/): alinhado com o nosso! (DRS)
- ES: SR (superintendência de saúde)

Indefinido:
- [AP](https://saude.portal.ap.gov.br/conteudo/institucional/regional-de-saude)
- [BA](http://www1.saude.ba.gov.br/mapa_bahia/VISAOMACRORREGIAOch.asp): referem-se a ambos! REGIÃO DE SAÚDE = MS/SAGE; Diretoria Regional (DRS) = nosso

## Importando pacotes

In [2]:
# ! pip3 install -U geobr
# ! pip3 install -U unidecode
import geobr
import pandas as pd
import numpy as np
from unidecode import unidecode

## Importando dados

### Nosso

In [3]:
# cnes_ref = pd.read_csv("http://datasource.coronacidades.org/br/cities/cnes", index_col=0)

# cnes_ref["state_num_id"] = cnes_ref["city_id"].astype(str).apply(lambda x : x[:2])

# region_id = cnes_ref[["state_num_id", "health_system_region"]]\
# .drop_duplicates()\
# .sort_values(["state_num_id", "health_system_region"])\
# .assign(health_region_id = lambda df: df["state_num_id"].astype(str) + df.groupby(["state_num_id"]).cumcount().astype(str))

# cols = ["country_name", "state_num_id", "state_id", "state_name", "city_id", 
#         "city_name", "health_system_region", "health_region_id"]

# cnes_ref = cnes_ref.merge(region_id, on=["health_system_region", "state_num_id"])[cols]
# cnes_ref.to_excel("br_health_region_drs_ours.xlsx")

df_ours = pd.read_csv("http://45.55.43.231:7000/br/places/ids", index_col=0)
df_ours.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country_name          5570 non-null   object 
 1   state_num_id          5570 non-null   int64  
 2   state_id              5570 non-null   object 
 3   state_name            5570 non-null   object 
 4   city_id               5570 non-null   int64  
 5   city_name             5570 non-null   object 
 6   health_system_region  5561 non-null   object 
 7   health_region_id      5561 non-null   float64
 8   data_last_refreshed   5570 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 435.2+ KB


In [13]:
df_ours.head()

,country_name,state_num_id,state_id,state_name,city_id,city_name,health_system_region,health_region_id,data_last_refreshed
index,,,,,,,,,
0,Brasil,12,AC,Acre,1200013,Acrelândia,Acre,120.0,2020-07-14 18:23:09
1,Brasil,12,AC,Acre,1200138,Bujari,Acre,120.0,2020-07-14 18:23:09
2,Brasil,12,AC,Acre,1200179,Capixaba,Acre,120.0,2020-07-14 18:23:09
3,Brasil,12,AC,Acre,1200344,Manoel Urbano,Acre,120.0,2020-07-14 18:23:09
4,Brasil,12,AC,Acre,1200385,Plácido de Castro,Acre,120.0,2020-07-14 18:23:09


### SAGE

In [11]:
df_sage = pd.read_csv("data/lista_dados_sage.csv", sep=";")
df_sage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   uf         5570 non-null   object
 1   cidade     5570 non-null   object
 2   ibge       5570 non-null   int64 
 3   regiao     5570 non-null   int64 
 4   no_regiao  5570 non-null   object
dtypes: int64(2), object(3)
memory usage: 217.7+ KB


In [12]:
df_sage.head()

,uf,cidade,ibge,regiao,no_regiao
0,AC,Acrelândia,120001,12002,Baixo Acre e Purus
1,AC,Assis Brasil,120005,12001,Alto Acre
2,AC,Brasiléia,120010,12001,Alto Acre
3,AC,Bujari,120013,12002,Baixo Acre e Purus
4,AC,Capixaba,120017,12002,Baixo Acre e Purus


### MS

In [50]:
df_ms = pd.read_csv("data/CIR-em-20-01-17.csv", sep=";", encoding="latin-1")
df_ms["CIRCOD"] = df_ms["CIRCOD"].astype(int)
df_ms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5568 entries, 0 to 5567
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   UF                          5568 non-null   object
 1   CIRCOD                      5568 non-null   int64 
 2   CIRNOME                     5568 non-null   object
 3   CODMUN                      5568 non-null   int64 
 4   Munic¡pio que compäe a CIR  5568 non-null   object
dtypes: int64(2), object(3)
memory usage: 217.6+ KB


In [51]:
df_ms.head()

,UF,CIRCOD,CIRNOME,CODMUN,Munic¡pio que compäe a CIR
0,AC,12001,Alto Acre,120005,Assis Brasil
1,AC,12001,Alto Acre,120010,Brasilia
2,AC,12001,Alto Acre,120025,Epitaciolndia
3,AC,12001,Alto Acre,120070,Xapuri
4,AC,12002,Baixo Acre e Purus,120001,Acrelndia


# Análise das regionalizações

### Quantas temos em cada?

In [52]:
print("SAGE: ", df_sage[["regiao"]].nunique()[0])
print("MS: ", df_ms[["CIRCOD"]].nunique()[0])
print("Nosso: ", df_ours[["health_region_id"]].nunique()[0])

SAGE:  450
MS:  437
Nosso:  358


### Quantos cada um tem por estado?

In [58]:
compare_n_state = pd.concat([
    df_sage.groupby("uf")["regiao"].nunique().rename("SAGE"), 
    df_ms.groupby("UF")["CIRCOD"].nunique().rename("MS"),
    df_ours.groupby("state_id")["health_region_id"].nunique().rename("Atual"),
], axis=1)

compare_n_state

,SAGE,MS,Atual
AC,3,3.0,3
AL,10,10.0,5
AM,9,9.0,7
AP,3,3.0,2
BA,28,28.0,31
CE,22,22.0,21
DF,1,NaN,1
ES,4,4.0,4
GO,18,18.0,16
MA,19,19.0,32


#### Quais estados é tudo diferente?

In [62]:
n = compare_n_state[(compare_n_state['SAGE'] != compare_n_state['MS']) 
                & (compare_n_state['SAGE'] != compare_n_state['Atual'])]

print("Número de UFs:", len(n))
n

Número de UFs: 1


,SAGE,MS,Atual
MG,89,77.0,28


#### Quais estados o nosso difere do SAGE?

In [63]:
n = compare_n_state[(compare_n_state['SAGE'] != compare_n_state['Atual'])]

print("Número de UFs:", len(n))
n

Número de UFs: 18


,SAGE,MS,Atual
AL,10,10.0,5
AM,9,9.0,7
AP,3,3.0,2
BA,28,28.0,31
CE,22,22.0,21
GO,18,18.0,16
MA,19,19.0,32
MG,89,77.0,28
MS,4,4.0,12
PA,13,13.0,23


#### Quais estados o nosso difere do MS?

In [66]:
n = compare_n_state[(compare_n_state['MS'] != compare_n_state['Atual'])]

print("Número de UFs - os mesmos de cima + DF:", len(n))
n

Número de UFs - os mesmos de cima + DF: 19


,SAGE,MS,Atual
AL,10,10.0,5
AM,9,9.0,7
AP,3,3.0,2
BA,28,28.0,31
CE,22,22.0,21
DF,1,NaN,1
GO,18,18.0,16
MA,19,19.0,32
MG,89,77.0,28
MS,4,4.0,12


### Quais regionais existem por estado?

Pode ser a mesma quantidade entre uma base e outra - mas são os mesmos nomes de regiões?

#### Lista por estado

In [117]:
uf = "CE"

In [118]:
l = df_sage.query(f"uf == '{uf}'")["no_regiao"].unique()
l.sort()

print("SAGE:  ", l)

SAGE:   ['10ª Região Limoeiro do Norte' '11ª Região Sobral' '12ª Região Acaraú'
 '13ª Região Tianguá' '14ª Região Tauá' '15ª Região Crateús'
 '16ª Região Camocim' '17ª Região Icó' '18ª Região Iguatú'
 '19ª Região Brejo Santo' '1ª Região Fortaleza' '20ª Região Crato'
 '21ª Região Juazeiro do Norte' '22ª Região Cascavel' '2ª Região Caucaia'
 '3ª Região Maracanaú' '4ª Região Baturité' '5ª Região Canindé'
 '6ª Região Itapipoca' '7ª Região Aracati' '8ª Região Quixadá'
 '9ª Região Russas']


In [119]:
l = df_ms.query(f"UF == '{uf}'")["CIRNOME"].unique()
l.sort()

print("MS:  ", l)

MS:   ['10¦ RegiÆo Limoeiro do Norte' '11¦ RegiÆo Sobral' '12¦ RegiÆo Acara£'
 '13¦ RegiÆo Tiangu\xa0' '14¦ RegiÆo Tau\xa0' '15¦ RegiÆo Crate£s'
 '16¦ RegiÆo Camocim' '17¦ RegiÆo Ic¢' '18¦ RegiÆo Iguat£'
 '19¦ RegiÆo Brejo Santo' '1¦ RegiÆo Fortaleza' '20¦ RegiÆo Crato'
 '21¦ RegiÆo Juazeiro do Norte' '22¦ RegiÆo Cascavel' '2¦ RegiÆo Caucaia'
 '3¦ RegiÆo Maracana£' '4¦ RegiÆo Baturit\x82' '5¦ RegiÆo Canind\x82'
 '6¦ RegiÆo Itapipoca' '7¦ RegiÆo Aracati' '8¦ RegiÆo Quixad\xa0'
 '9¦ RegiÆo Russas']


In [120]:
l = df_ours.query(f"state_id == '{uf}'")["health_system_region"].unique()
l.sort()

print("Atual:  ", l)

Atual:   ['Acaraú' 'Aracati' 'Baturité' 'Brejo Santo' 'Camocim' 'Canindé' 'Caucaia'
 'Crateús' 'Crato' 'Fortaleza' 'Icó' 'Iguatu' 'Itapipoca'
 'Juazeiro do Norte' 'Limoeiro do Norte' 'Maracanaú' 'Quixadá' 'Russas'
 'Sobral' 'Tauá' 'Tianguá']


### Quais municípios estão em quais regionais?